In [41]:
from rooms_mdp import Mdp
import numpy as np
import sys

env = Mdp()
env.regions = []
region = []
for elem in [list(range(10*i, 10*i+5)) for i in range(5)]:
    region.extend(elem)
env.regions.append(region)
region = []
for elem in [list(range(10*i, 10*i+5)) for i in range(5, 10)]:
    region.extend(elem)
env.regions.append(region)
region = []
for elem in [list(range(10*i+5, 10*i+10)) for i in range(5)]:
    region.extend(elem)
env.regions.append(region)
region = []
for elem in [list(range(10*i, 10*i+5)) for i in range(5, 10)]:
    region.extend(elem)
env.regions.append(region)

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [50]:
def make_not_region(region, S):
    not_region = []
    for elem in range(S):
        if elem not in region:
            not_region.append(elem)
    return not_region

def make_kernel(region, P, not_region):
    kernel = []
    for state in region:
        connected_states = list(np.where(env.P[:, state] != 0)[1])
        if not len(intersection(connected_states, not_region)):
            kernel.append(state)
    return sorted(kernel)

def update_peripheries(U, region, kernel):
    for elem in region:
        if elem not in kernel:
            U.append(elem)
    return U

def local_transition(kernel, U, P):
    state_space = kernel+U
    A = P.shape[0]
    S = len(state_space)
    local_P = np.zeros((A, S, S))
    for a in range(A):
        for s1 in range(S):
            for s2 in range(S):
                if s1 in U:
                    local_P[a, s1, s1] = 1
                else:
                    local_P[a, s1, s2] = P[a, s1, s2]
    return local_P

def local_reward(kernel, U, R, beta):
    state_space = kernel+U
    A = R.shape[1]
    S = len(state_space)
    local_R = np.zeros((A, S, S))
    for a in range(A):
        for s1 in range(S):
            for s2 in range(S):
                if s1 in kernel:
                    if s2 in kernel:
                        local_R[a, s1, s2] = R[a, s1, s2]
    return local_R

class DeanLinDecomposition:
    def __init__(self, env=env) -> None:
        self.env = env
        self.S = self.env.S
        self.A = self.env.A
        self.P = self.env.P
        self.R = self.env.R
        self.regions = self.env.regions

        self.make_kernels()
        self.make_peripheries()
        self.m = len(self.U)
        self.beta = [0 for _ in range(self.m+1)]

        self.make_local_transitions()
        self.make_local_rewards()


    def make_kernels(self):
        self.kernels = []
        for region in self.regions:
            not_region = make_not_region(region, self.S)
            self.kernels.append(make_kernel(region, self.P, not_region))
        
    def make_peripheries(self):
        self.U = []
        for region_index, kernel in enumerate(self.kernels):
            self.U = update_peripheries(self.U, self.regions[region_index], kernel)

    def make_local_transitions(self):
        self.local_transitions = []
        for kernel in self.kernels:
            self.local_transitions.append(local_transition(kernel, self.U, self.P))

    def make_local_rewards(self):
        self.local_rewards = []
        for kernel in self.kernels:
            self.local_rewards.append(local_reward(kernel, self.U, self.R, self.beta))

sol = DeanLinDecomposition()


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed